In [2]:
import xgboost as xg
print(xg.__version__)

1.0.1


In [3]:
import pandas as pd

In [4]:
dataset=pd.read_csv("insurance_pre.csv")

In [5]:
dataset

,age,sex,bmi,children,smoker,charges
0,19,female,27.900,0,yes,16884.92400
1,18,male,33.770,1,no,1725.55230
2,28,male,33.000,3,no,4449.46200
3,33,male,22.705,0,no,21984.47061
4,32,male,28.880,0,no,3866.85520
...,...,...,...,...,...,...
1333,50,male,30.970,3,no,10600.54830
1334,18,female,31.920,0,no,2205.98080
1335,18,female,36.850,0,no,1629.83350
1336,21,female,25.800,0,no,2007.94500


In [6]:
dataset=pd.get_dummies(dataset,drop_first=True)

In [7]:
dataset

,age,bmi,children,charges,sex_male,smoker_yes
0,19,27.900,0,16884.92400,0,1
1,18,33.770,1,1725.55230,1,0
2,28,33.000,3,4449.46200,1,0
3,33,22.705,0,21984.47061,1,0
4,32,28.880,0,3866.85520,1,0
...,...,...,...,...,...,...
1333,50,30.970,3,10600.54830,1,0
1334,18,31.920,0,2205.98080,0,0
1335,18,36.850,0,1629.83350,0,0
1336,21,25.800,0,2007.94500,0,0


In [8]:
dataset.columns

Index(['age', 'bmi', 'children', 'charges', 'sex_male', 'smoker_yes'], dtype='object')

In [9]:
independent=dataset.iloc[:, [0, 1, 2, 4, 5]]  #independent would contain a DataFrame with all rows, 
#but only the columns specified by their integer positions: the first, second, third, fifth, and sixth columns of dataset.

In [10]:
independent

,age,bmi,children,sex_male,smoker_yes
0,19,27.900,0,0,1
1,18,33.770,1,1,0
2,28,33.000,3,1,0
3,33,22.705,0,1,0
4,32,28.880,0,1,0
...,...,...,...,...,...
1333,50,30.970,3,1,0
1334,18,31.920,0,0,0
1335,18,36.850,0,0,0
1336,21,25.800,0,0,0


In [11]:
dependent=dataset[['charges']]

In [12]:
dependent

,charges
0,16884.92400
1,1725.55230
2,4449.46200
3,21984.47061
4,3866.85520
...,...
1333,10600.54830
1334,2205.98080
1335,1629.83350
1336,2007.94500


In [29]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor
param_grid={'max_depth':[5,7],'n_estimators':[10,100,200]}
grid=GridSearchCV(XGBRegressor(),param_grid,refit=True,verbose=3,n_jobs=-1) #grid will be an object that, when fitted with data, will perform a grid search over the specified parameter grid for an XGBoost regressor.
#After fitting, it will have attributes like best_params_, best_score_, and best_estimator_, which you can use to access information about the best performing model and its hyperparameters.
grid.fit(independent,dependent)

Fitting 3 folds for each of 6 candidates, totalling 18 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of  18 | elapsed:    0.0s remaining:    0.5s
[Parallel(n_jobs=-1)]: Done  10 out of  18 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed:    2.7s finished


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=XGBRegressor(base_score=None, booster=None,
                                    colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, gamma=None,
                                    gpu_id=None, importance_type='gain',
                                    interaction_constraints=None,
                                    learning_rate=None, max_delta_step=None,
                                    max_depth=None, min_child_weight=None,
                                    missing=nan, monotone_cons...
                                    num_parallel_tree=None,
                                    objective='reg:squarederror',
                                    random_state=None, reg_alpha=None,
                                    reg_lambda=None, scale_pos_weight=None,
                                    subsample=None, tree

In [30]:
print(independent.columns)

Index(['age', 'bmi', 'children', 'sex_male', 'smoker_yes'], dtype='object')


In [31]:
re=grid.cv_results_
print(re) #Printing grid.cv_results_ will provide detailed information about the cross-validation results 
#for each combination of hyperparameters

{'mean_fit_time': array([0.03800726, 0.09945432, 0.20509744, 0.02430018, 0.13853383,
       0.21860115]), 'std_fit_time': array([0.0220982 , 0.00172825, 0.00537861, 0.00975487, 0.00308162,
       0.02360933]), 'mean_score_time': array([0.        , 0.        , 0.00663853, 0.        , 0.00734933,
       0.00947515]), 'std_score_time': array([0.        , 0.        , 0.00053139, 0.        , 0.00165772,
       0.0071106 ]), 'param_max_depth': masked_array(data=[5, 5, 5, 7, 7, 7],
             mask=[False, False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_n_estimators': masked_array(data=[10, 100, 200, 10, 100, 200],
             mask=[False, False, False, False, False, False],
       fill_value='?',
            dtype=object), 'params': [{'max_depth': 5, 'n_estimators': 10}, {'max_depth': 5, 'n_estimators': 100}, {'max_depth': 5, 'n_estimators': 200}, {'max_depth': 7, 'n_estimators': 10}, {'max_depth': 7, 'n_estimators': 100}, {'max_depth': 7, 'n_es

In [32]:
print("The R2_score value for the best parameter{}:".format(grid.best_params_))

The R2_score value for the best parameter{'max_depth': 5, 'n_estimators': 10}:


In [33]:
print("Best R^2 score:", grid.best_score_)

Best R^2 score: 0.8506235047990774


In [34]:
table=pd.DataFrame.from_dict(re)

In [35]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,0.038007,0.022098,0.000000,0.000000,5,10,"{'max_depth': 5, 'n_estimators': 10}",0.864072,0.851126,0.836672,0.850624,0.011191,1
1,0.099454,0.001728,0.000000,0.000000,5,100,"{'max_depth': 5, 'n_estimators': 100}",0.823337,0.802108,0.808420,0.811288,0.008901,3
2,0.205097,0.005379,0.006639,0.000531,5,200,"{'max_depth': 5, 'n_estimators': 200}",0.807156,0.781961,0.797263,0.795460,0.010364,6
3,0.024300,0.009755,0.000000,0.000000,7,10,"{'max_depth': 7, 'n_estimators': 10}",0.842092,0.838152,0.824992,0.835079,0.007312,2
4,0.138534,0.003082,0.007349,0.001658,7,100,"{'max_depth': 7, 'n_estimators': 100}",0.808823,0.801813,0.799231,0.803289,0.004053,4
5,0.218601,0.023609,0.009475,0.007111,7,200,"{'max_depth': 7, 'n_estimators': 200}",0.806103,0.797315,0.797188,0.800202,0.004173,5


In [36]:
import pickle
import xgboost

# Trained model object (assuming it's already trained and stored in the 'regressor' variable)
regressor = xgboost.XGBRegressor(objective='reg:linear', n_estimators=10)
regressor.fit(independent,dependent)

# Save the trained model to a file
filename = "finalized_model_XGBM.sav"
pickle.dump(regressor, open(filename, 'wb'))

[14:08:29] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.0.0/src/objective/regression_obj.cu:167: reg:linear is now deprecated in favor of reg:squarederror.


In [37]:
import pickle
import pandas as pd

# Load the trained model
loaded_model = pickle.load(open("finalized_model_XGBM.sav", 'rb'))

# Define the prediction input
prediction = [[30, 22, 1, 1, 0]]
prediction_df = pd.DataFrame(prediction, columns=['age', 'bmi', 'children', 'sex_male', 'smoker_yes'])

# Make the prediction
result = loaded_model.predict(prediction_df)
print(result)

[14:08:44] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.0.0/src/objective/regression_obj.cu:167: reg:linear is now deprecated in favor of reg:squarederror.
[5222.6885]
